In [1]:
import numpy as np
from tensorflow import keras

In [2]:
def save_keras_model(model, filename="model.svg", expand_nested=False, **kwargs):
    with open(filename, "wb+") as f:
        f.write(
            keras.utils.model_to_dot(
                model=model,
                show_shapes=True,
                show_dtype=True,
                show_layer_names=True,
                expand_nested=expand_nested,
                **kwargs
            ).create_svg()
        )

In [3]:
INPUT_SHAPE = (224, 224, 3)
N_CLASSES = 399

In [4]:
# m = keras.applications.EfficientNetB0(include_top=True, weights=None)

In [5]:
input_msg = keras.layers.Input(
    shape=INPUT_SHAPE,
    dtype="uint8",
    name="input_msg",
)
input_year = keras.layers.Input(shape=(1,), dtype="float32", name="input_year")


x = keras.applications.EfficientNetB0(
    include_top=False,
    weights=None,
)(input_msg)

features_msg = keras.layers.GlobalAveragePooling2D(name="features_msg")(x)
features = keras.layers.Concatenate(axis=1)([features_msg, input_year])

output_1 = keras.layers.Dense(N_CLASSES, name="out_classes", activation="sigmoid")(
    features
)

In [6]:
m = keras.Model(inputs=[input_msg, input_year], outputs=[output_1])

In [30]:
2 * np.pi

6.283185307179586

In [5]:
import tensorflow.keras.backend as K


class SinCos(keras.layers.Layer):
    def __init__(self, val_range=[0, 1], **kwargs):
        super(SinCos, self).__init__(**kwargs)
        self.val_range = val_range

    def call(self, inputs, training=None):

        norm_inputs = (
            (inputs - self.val_range[0])
            / (self.val_range[1] - self.val_range[0])
            * 2
            * np.pi
        )

        return K.concatenate(
            [K.sin(norm_inputs), K.cos(norm_inputs)],
        )

    def get_config(self):
        return {"val_range": self.val_range}

In [6]:
i1 = keras.layers.Input(shape=(1,), dtype="float32", name="input_month")
x = SinCos(val_range=[1, 12])(i1)
m = keras.Model(inputs=[i1], outputs=[x])

In [7]:
# m.build((1,)

In [8]:
save_keras_model(m)

In [9]:
r = np.array(m.predict(np.arange(1, 13)))

In [10]:
r

array([[ 0.0000000e+00,  1.0000000e+00],
       [ 5.4064083e-01,  8.4125352e-01],
       [ 9.0963203e-01,  4.1541496e-01],
       [ 9.8982143e-01, -1.4231496e-01],
       [ 7.5574946e-01, -6.5486085e-01],
       [ 2.8173238e-01, -9.5949304e-01],
       [-2.8173277e-01, -9.5949292e-01],
       [-7.5574976e-01, -6.5486056e-01],
       [-9.8982149e-01, -1.4231454e-01],
       [-9.0963197e-01,  4.1541508e-01],
       [-5.4064047e-01,  8.4125376e-01],
       [ 6.5168274e-07,  1.0000000e+00]], dtype=float32)

In [11]:
v = 12
np.sin((v - 1) / 11 * 2 * np.pi)
np.cos((v - 1) / 11 * 2 * np.pi)

1.0

In [13]:
m.get

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.